# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data=pd.read_csv('city_csv')
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Klaksvík,62.2266,-6.5890,40.12,76,100,20.60,FO,1621382675
1,Biltine,14.5333,20.9167,84.78,12,6,3.18,TD,1621382675
2,Fairbanks,64.8378,-147.7164,62.56,32,75,12.66,US,1621382652
3,Bredasdorp,-34.5322,20.0403,53.26,73,59,2.71,ZA,1621382675
4,Bathsheba,13.2167,-59.5167,81.21,73,40,14.61,BB,1621382676


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
humidity = weather_data["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_data[["Lat", "Lng"]]

In [7]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
plt.savefig('heat map new.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
new_weather_df = weather_data.loc[(weather_data['Max Temp'] < 80 )&(weather_data['Max Temp']> 70)&(weather_data['Cloudiness'] ==0)&(weather_data['Wind Speed']<10),: ]
new_weather_df.reset_index(inplace=True)
new_weather_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,41,Al Bardīyah,31.7561,25.0865,71.15,50,0,4.36,LY,1621382684
1,218,Três Lagoas,-20.7511,-51.6783,70.23,45,0,5.68,BR,1621382725
2,305,Coxim,-18.5067,-54.7600,76.77,31,0,2.98,BR,1621382746
3,406,Kaka,37.3482,59.6143,76.26,16,0,4.61,TM,1621382529
4,495,Vostok,46.4856,135.8833,70.70,26,0,8.34,RU,1621382789


In [5]:
del new_weather_df['index']

In [6]:
new_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Al Bardīyah,31.7561,25.0865,71.15,50,0,4.36,LY,1621382684
1,Três Lagoas,-20.7511,-51.6783,70.23,45,0,5.68,BR,1621382725
2,Coxim,-18.5067,-54.7600,76.77,31,0,2.98,BR,1621382746
3,Kaka,37.3482,59.6143,76.26,16,0,4.61,TM,1621382529
4,Vostok,46.4856,135.8833,70.70,26,0,8.34,RU,1621382789


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = new_weather_df
hotel_df['Hotel_name']=''
hotel_df.head()

C:\Users\jacke\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_name
0,Al Bardīyah,31.7561,25.0865,71.15,50,0,4.36,LY,1621382684,
1,Três Lagoas,-20.7511,-51.6783,70.23,45,0,5.68,BR,1621382725,
2,Coxim,-18.5067,-54.7600,76.77,31,0,2.98,BR,1621382746,
3,Kaka,37.3482,59.6143,76.26,16,0,4.61,TM,1621382529,
4,Vostok,46.4856,135.8833,70.70,26,0,8.34,RU,1621382789,


In [13]:
hotel = []

for i in range(len(new_weather_df)):
    lat = new_weather_df.loc[i]['Lat']
    long = new_weather_df.loc[i]['Lng']
    
    print(i)

0
1
2
3
4
5
6


In [14]:
hotel = []

for i in range(len(new_weather_df)):
    lat = new_weather_df.loc[i]['Lat']
    long = new_weather_df.loc[i]['Lng']

# set up a parameters dictionary
    params = {
    "location": f'{lat},{long}',
    "radius": 5000,
    "type": "hotel",
    "key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_weather= response.json()
    
    try:
        hotel.append(response_weather['results'][0]['name'])
    except:
        hotel.append("")
hotel_df["Hotel_Name"] = hotel
hotel_df = hotel_df.dropna(how='any')

hotel_df.head()

        




C:\Users\jacke\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_name,Hotel_Name
0,Al Bardīyah,31.7561,25.0865,71.15,50,0,4.36,LY,1621382684,,Bardiyah
1,Três Lagoas,-20.7511,-51.6783,70.23,45,0,5.68,BR,1621382725,,Três Lagoas
2,Coxim,-18.5067,-54.7600,76.77,31,0,2.98,BR,1621382746,,Coxim
3,Kaka,37.3482,59.6143,76.26,16,0,4.61,TM,1621382529,,Kaakhka
4,Vostok,46.4856,135.8833,70.70,26,0,8.34,RU,1621382789,,Vostok


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
fig= gmaps.figure()
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))